# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [13]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import ttest_1samp
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [7]:
# Run this code:
salaries = pd.read_csv('C:\\Users\\Francesco Corda\\Documents\\GitHub\\lab-hypothesis-testing-en\\data\\Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [8]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [10]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [20]:
# create subset of data
hourly_salary = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate']


#compute the test statistics and the p-value
#H0 = 30 / H1 != 30
test_statistic, p_value = ttest_1samp(hourly_salary,30)

hourly_mean = hourly_salary.mean()

print(f'Test Statistic: {test_statistic}')
print(f'P-value: {p_value}')
print(f'Mean: {hourly_mean}')

#significance level
alpha = 0.05

if alpha < p_value:
    print("Reject the null hypothesis: The mean hourly wage is significantly different from $30/hr.")
else:
    print("Fail to reject the null hypothesis: The mean hourly wage is not significantly different from $30/hr.")



Test Statistic: 20.6198057854942
P-value: 4.3230240486229894e-92
Mean: 32.78855771628024
Fail to reject the null hypothesis: The mean hourly wage is not significantly different from $30/hr.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [21]:
# Your code here

# Calculate the sample mean and standard error
mean_hourly_rate = np.mean(hourly_salary)
sem_hourly_rate = sem(hourly_salary)

# Calculate the confidence interval
confidence_level = 0.95
degrees_of_freedom = len(hourly_salary) - 1
confidence_interval = t.interval(confidence_level, degrees_of_freedom, loc=mean_hourly_rate, scale=sem_hourly_rate)

print(f"95% Confidence Interval for the mean hourly wage: {confidence_interval}")

95% Confidence Interval for the mean hourly wage: (32.52345834488425, 33.05365708767623)


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [25]:
from statsmodels.stats.proportion import proportions_ztest

# Count the number of hourly workers
num_hourly_workers = hourly_salary.shape[0]

# Count the total number of workers
total_workers = salaries.shape[0]

# Perform the proportions z-test
count = num_hourly_workers
nobs = total_workers
value = 0.25  # hypothesized proportion

# Perform the one-sample z-test for proportions
stat, p_value = proportions_ztest(count, nobs, value)

# Print the test statistic and p-value
print(f'Test Statistic: {stat}')
print(f'P-value: {p_value}')

# Determine if we reject or fail to reject the null hypothesis
alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject the null hypothesis: The proportion of hourly workers is significantly different from 25%.")
else:
    print("Fail to reject the null hypothesis: The proportion of hourly workers is not significantly different from 25%.")



Test Statistic: -3.5099964213703005
P-value: 0.0004481127249057967
Reject the null hypothesis: The proportion of hourly workers is significantly different from 25%.


0.24175029382515142